In [187]:
%matplotlib inline
import pandas as pd
import datetime as dt

In [188]:
# Data: MovieLens
# Read in Movies (m) and Ratings (r) from MovieLens Data
dataDir = '/Users/justinbarton/Documents/Personal/DataScienceIntensive/Data'
m = pd.read_csv(dataDir + '/MovieLens/ml-20m/movies.csv')
r = pd.read_csv(dataDir + '/MovieLens/ml-20m/ratings.csv')

In [189]:
# Data: MovieLens
# Find the average movie rating and store in 'mr'
mr = pd.DataFrame(r[['movieId','rating']].groupby('movieId').mean())
mr.head()

,rating
movieId,
1,3.921240
2,3.211977
3,3.151040
4,2.861393
5,3.064592


In [190]:
# Data: MovieLens
# Count the number of ratings and store in 'mc'. Sort by highest number of ratings forst
mc = pd.DataFrame(r[['movieId','rating']].groupby('movieId').size(), columns=['num_ratings']).sort_values(by='num_ratings', ascending=0)
mc.head()

,num_ratings
movieId,
296,67310
356,66172
318,63366
593,63299
480,59715


In [191]:
# Data: MovieLens
# Show all movies, number of ratings 'num_ratings' and average rating 'rating'
mrmc = mc.merge(mr, right_index=1, left_index=1)
mrmc.head()

,num_ratings,rating
movieId,,
1,49695,3.921240
2,22243,3.211977
3,12735,3.151040
4,2756,2.861393
5,12161,3.064592


In [192]:
# Data: MovieLens
# Show the top movies if sorted by average rating, then number of ratings
ms = (m.merge(mrmc,how='left', left_on='movieId',right_index=1).groupby(['title','genres']).mean().
    sort_values(['rating','num_ratings'], ascending=[0,0]))
ms.head(10)

,,movieId,num_ratings,rating
title,genres,,,
Catastroika (2012),Documentary,108527,2.0,5.0
Consuming Kids: The Commercialization of Childhood (2008),Documentary,103871,2.0,5.0
1971 (2014),Documentary,114254,1.0,5.0
A Blank on the Map (1971),Documentary,129478,1.0,5.0
A Gun for Jennifer (1997),Crime|Drama|Thriller,129295,1.0,5.0
A Night for Dying Tigers (2010),Drama,121039,1.0,5.0
Abendland (2011),Documentary,101292,1.0,5.0
Afstiros katallilo (2008),Comedy,129243,1.0,5.0
Al otro lado (2004),Drama,40404,1.0,5.0


In [193]:
# Data: MovieLens
# Find and show the top movies if sorted by average rating, then number of ratings, ...
# for only those that have > 100 ratings, store in 'top_better'
top_better = m.merge(mrmc[mrmc.num_ratings>100],how='inner', left_on='movieId',right_index=1) \
                .groupby(['title','genres']).mean() \
              .sort_values(['rating','num_ratings'], ascending=[0,0])
top_better[top_better.rating.notnull()].head()
top_better.reset_index(inplace=True)
len(top_better)
#top_better

8532

In [194]:
# Data: MovieLens
# Separate the title and the year in separate columns for MovieLen titles
top_better['year'] = top_better.title.str.extract('[(](\d{4})[)]') 
top_better['title'] = top_better.title.str.replace(r' [(]\d{4}[)]', '')
top_better['year'].fillna('0', inplace=True)
top_better['year'] = top_better['year'].astype(int)
top_better.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8532 entries, 0 to 8531
Data columns (total 6 columns):
title          8532 non-null object
genres         8532 non-null object
movieId        8532 non-null int64
num_ratings    8532 non-null int64
rating         8532 non-null float64
year           8532 non-null int64
dtypes: float64(1), int64(3), object(2)
memory usage: 400.0+ KB


/Users/justinbarton/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:3: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  app.launch_new_instance()


In [195]:
# Data: MovieLens
# Move the 'The' to the start of the film title for MovieLens titles
df = top_better
df['has_the'] = df.title.str.contains(', The')
df.title = df.title.str.replace(r', The', '')
df.ix[df.has_the, 'title'] = 'The ' + df.ix[df.has_the, 'title']
df[df.has_the].head()

,title,genres,movieId,num_ratings,rating,year,has_the
0,The Shawshank Redemption,Crime|Drama,318,63366,4.446990,1994,True
1,The Godfather,Crime|Drama,858,41355,4.364732,1972,True
2,The Usual Suspects,Crime|Mystery|Thriller,50,47006,4.334372,1995,True
4,The Godfather: Part II,Crime|Drama,1221,27398,4.275641,1974,True
12,The Third Man,Film-Noir|Mystery|Thriller,1212,6565,4.246002,1949,True


In [196]:
# Data: MovieLens
# Find and show from 'top better' movies, only those made on or after 1965
top_better_new = top_better[top_better.year >= 1965]
#len(top_better_new)
top_better_new.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7640 entries, 0 to 8531
Data columns (total 7 columns):
title          7640 non-null object
genres         7640 non-null object
movieId        7640 non-null int64
num_ratings    7640 non-null int64
rating         7640 non-null float64
year           7640 non-null int64
has_the        7640 non-null bool
dtypes: bool(1), float64(1), int64(3), object(2)
memory usage: 425.3+ KB


In [197]:
# Data: Excel - My Movie Ratings
# Read in the films that I've already reviewed
pd.set_option('display.max_rows', 5000)
jb = pd.read_excel('output_JB1.xlsx')
jb.movieId = jb.movieId.astype(int)
jb = jb.merge(mrmc, left_on='movieId', right_index=1)
jb['reviewed'] = jb['out_of_10'].bfill().notnull() # Flag all of the films I've reviwed, so that I don't have to do it again
jb = jb[jb['reviewed']]
jb_watched = jb[jb['out_of_10'].notnull()]
jb_watched.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 274 entries, 256 to 2258
Data columns (total 6 columns):
movieId        274 non-null int64
title          274 non-null object
out_of_10      274 non-null float64
num_ratings    274 non-null int64
rating         274 non-null float64
reviewed       274 non-null bool
dtypes: bool(1), float64(2), int64(2), object(1)
memory usage: 13.1+ KB


In [198]:
# Data: OMDB
# Read in data from from omdb Data
dataDir = '/Users/justinbarton/Documents/Personal/DataScienceIntensive/Data'
om = pd.read_csv(dataDir + '/omdb/all_movies.csv', error_bad_lines=False)
od = pd.read_csv(dataDir + '/omdb/movie_details.csv')
ol = pd.read_csv(dataDir + '/omdb/movie_languages.csv')
ot = pd.read_csv(dataDir + '/omdb/trailers.csv')
op = pd.read_csv(dataDir + '/omdb/all_people.csv')
oj = pd.read_csv(dataDir + '/omdb/job_names.csv')
oc = pd.read_csv(dataDir + '/omdb/all_casts.csv', error_bad_lines=False)

# Find all jobs for english speaking films
jobs = oj[oj.language_iso_639_1=='en'].merge(oc, left_on='job_id', right_on='job_id')
#most_common_jobs = jobs.groupby('name').size().sort_values(ascending=0)
#'Actor', 'Director', 'Screenplay', 'Producer', 'Director of Photography', 'Editor', 'Original Music Composer'
#'Editor', 'Original Music Composer', 'Executive Producer', 'Music'

# Find all directors and their movies
directorID = oj[(oj.name=='Director')&(oj.language_iso_639_1=='en')].job_id.iloc[0]
directors_movies = jobs[jobs.name=='Director'].merge(op, left_on='person_id', right_on='id')[['movie_id','person_id','name_y']]
directors_movies.head()

Skipping line 6074: expected 4 fields, saw 5
Skipping line 28493: expected 4 fields, saw 5

Skipping line 39995: expected 5 fields, saw 6
Skipping line 110384: expected 5 fields, saw 6

Skipping line 187226: expected 5 fields, saw 6
Skipping line 237735: expected 5 fields, saw 9
Skipping line 237736: expected 5 fields, saw 9

Skipping line 332128: expected 5 fields, saw 8
Skipping line 332424: expected 5 fields, saw 7
Skipping line 348703: expected 5 fields, saw 6
Skipping line 365812: expected 5 fields, saw 6

Skipping line 655284: expected 5 fields, saw 6
Skipping line 655285: expected 5 fields, saw 6

Skipping line 679612: expected 5 fields, saw 6



,movie_id,person_id,name_y
0,11,1,George Lucas
1,636,1,George Lucas
2,838,1,George Lucas
3,1893,1,George Lucas
4,1894,1,George Lucas


In [199]:
# Data: OMDB
# Get the Year from the OMDB data
om.date.astype(str, inplace=True) # Convert dates to string
om.loc[om.date == '\N'] = '' # Remove null dates '/N'
om['year'] = pd.to_datetime(om.date.astype(str), format='%Y-%m-%d') # Get the date format
om['year'] = om['year'].dt.year.fillna(0).astype(int) # Extract the year as an int, from the date
om.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39726 entries, 0 to 39725
Data columns (total 5 columns):
id           39726 non-null object
name         39726 non-null object
parent_id    39726 non-null object
date         39726 non-null object
year         39726 non-null int64
dtypes: int64(1), object(4)
memory usage: 1.5+ MB


In [257]:
# Data: MovieLens & OMDB
# Merge the MovieLens and OMDB data into one dataframe called 'films' (merging by movie title and year)
# (This is a very basic way to join movie titles together, but will suffice for proof of concept.
# Some examples of incorrect mismatches to be studied later include:
# 'City of God (Cidade de Deus)' in MovieLens and 'City of God' in OMDB 
# 'One Flew Over the Cuckoo's Nest' in MovieLens and 'One Flew Over The Cuckoo's Nest' in OMDB
# 'Star Wars: Episode IV - A New Hope' in MovieLens and 'Star Wars: Episode IV – A New Hope' in OMDB)
df = top_better_new.merge(om, left_on=['title', 'year'], right_on=['name', 'year'], how='left')
df = df[df.name.notnull()]
films = df
films.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5046 entries, 0 to 7650
Data columns (total 11 columns):
title          5046 non-null object
genres         5046 non-null object
movieId        5046 non-null int64
num_ratings    5046 non-null int64
rating         5046 non-null float64
year           5046 non-null int64
has_the        5046 non-null bool
id             5046 non-null object
name           5046 non-null object
parent_id      5046 non-null object
date           5046 non-null object
dtypes: bool(1), float64(1), int64(3), object(6)
memory usage: 438.6+ KB


In [258]:
# Data: MovieLens & OMDB
# Remove all of the films that don't have an English language format
#df = films
df = films
df = df.merge(ol, left_on='id', right_on='movie_id')
df = df[df.language_iso_639_1 == 'en']
films = df
films.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4859 entries, 0 to 5852
Data columns (total 13 columns):
title                 4859 non-null object
genres                4859 non-null object
movieId               4859 non-null int64
num_ratings           4859 non-null int64
rating                4859 non-null float64
year                  4859 non-null int64
has_the               4859 non-null bool
id                    4859 non-null object
name                  4859 non-null object
parent_id             4859 non-null object
date                  4859 non-null object
movie_id              4859 non-null int64
language_iso_639_1    4859 non-null object
dtypes: bool(1), float64(1), int64(4), object(7)
memory usage: 498.2+ KB


In [250]:
# Data: IMDB
# Read in IMDB Data
cast = pd.read_csv(dataDir +'/IMDB/cast.csv')
titles = pd.read_csv(dataDir +'/IMDB/titles.csv')

# Create data frame with titles and films, it and with id 'index'
titles.reset_index(inplace=True)
titles_cast = cast.merge(titles)

titles_cast.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3475509 entries, 0 to 3475508
Data columns (total 7 columns):
title        object
year         int64
name         object
type         object
character    object
n            float64
index        int64
dtypes: float64(1), int64(2), object(4)
memory usage: 212.1+ MB


In [259]:
# Data: MovieLens & OMDB & IMDB
# Connect IMDB data to 'films' in order to call upon cast list
# (similarly to above, 'title' and 'year' here do not provide a full-proof matching system between data sets,
# will need to analyse this further in future versions.. For example:
# 'V for Vendetta' is a 2006 movie in MovieLens and OMDB, but its a 2005 movie in IMDB,
# 'Batman: The Dark Knight Returns, Part 2' for MovieLens/OMDB and 'The Dark Knight: Knightfall - Part Two' in IMDB
# 'Birdman' in MovieLens and OMDB and 'Birdman or (The Unexpected Virtue of Ignorance)' in IMDB
df = films
imdb_connector_fields = ['title', 'year']
df = df.merge((titles_cast[['index']+imdb_connector_fields]).drop_duplicates(), left_on=['title', 'year'], 
              right_on = imdb_connector_fields, how='left')
df.info()
df = df[df['index'].notnull()]
films = df
films.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4859 entries, 0 to 4858
Data columns (total 14 columns):
title                 4859 non-null object
genres                4859 non-null object
movieId               4859 non-null int64
num_ratings           4859 non-null int64
rating                4859 non-null float64
year                  4859 non-null int64
has_the               4859 non-null bool
id                    4859 non-null object
name                  4859 non-null object
parent_id             4859 non-null object
date                  4859 non-null object
movie_id              4859 non-null int64
language_iso_639_1    4859 non-null object
index                 4555 non-null float64
dtypes: bool(1), float64(2), int64(4), object(7)
memory usage: 536.2+ KB


In [ ]:
# Output spreadsheet of films for users to review...

In [220]:
# Data: MovieLens & OMDB & Excel(ratings)
# Find all the films I've watched that are in the films dataframe
jb_films = jb_watched.merge(films, left_on='movieId', right_on='movieId', how='left')
jb_films = jb_films[jb_films.title_y.notnull()]

In [244]:
# Find my preferred directors
df = jb_films
jb_directors = df[df.out_of_10 >= 7].merge(directors_movies, left_on='id', right_on='movie_id')
director_counts = pd.DataFrame(jb_directors.groupby(['name_y']).size(), columns=['director_count'])
jb_directors['director_count'] = jb_directors.merge(director_counts, left_on='name_y', right_index=True)['director_count']

fields_desired = ['title_y', 'out_of_10', 'name_y', 'director_count', 'num_ratings_y','rating_y']

jb_directors.sort_values(by=['director_count','name_y'], ascending=[False, True])[fields_desired]

,title_y,out_of_10,name_y,director_count,num_ratings_y,rating_y
15,The Dark Knight Rises,8.0,Christopher Nolan,5,4770.0,4.000210
20,Batman Begins,8.0,Christopher Nolan,5,18686.0,3.970754
29,The Prestige,9.0,Christopher Nolan,5,11269.0,4.042195
32,Memento,9.0,Christopher Nolan,5,30443.0,4.178547
36,The Dark Knight,8.0,Christopher Nolan,5,20438.0,4.220129
9,Raging Bull,7.0,Martin Scorsese,5,9711.0,4.083411
35,The Departed,9.0,Martin Scorsese,5,14309.0,4.109931
39,Taxi Driver,7.0,Martin Scorsese,5,24481.0,4.110576
40,Goodfellas,8.0,Martin Scorsese,5,26406.0,4.183633
61,Shutter Island,9.0,Martin Scorsese,5,5930.0,3.904469


In [ ]:
# Take the above and create a generic function 'my preferred people' tables, which could be directors, screenplay, music, billed actors

In [ ]:
# Find my preferred screenplay

In [ ]:
# Find my preferred actors/actresses (that are at least credited in that film)

In [ ]:
# Find simliar users

In [ ]:
# To be used for generating the surveys for people
#df = top_better

#top500 = df.head(500)
#top500 = top500.reindex(pd.np.random.permutation(top500.index))
#next4500 = df.head(5000).tail(4500)
#next4500 = next4500.reindex(pd.np.random.permutation(next4500.index))
#output = top500.append(next4500)
#output.head()

In [ ]:
#import sys  
#reload(sys)  
#sys.setdefaultencoding('utf8')

# Save output to spreadsheet for each person
#output[['movieId','title']].to_excel('output.xlsx','Sheet1')